In [ ]:
'''
==========================================
 Title:  Backtest
 Author: Haoli Yin / Nano1337
 Date:   11 March 2021
==========================================
'''
!pip install stable-baselines3  

In [ ]:
import numpy as np

file = np.load("weekdata.npy")

# reassemble project list from numpy file
data = []
for i in range(len(file)):
  project = file[i][:2].tolist()
  project += list(map(int, file[i][2:3]))
  project.append(list(map(int, file[i][3:10].tolist()))) # write frequencies
  project.append(list(map(int, file[i][10:].tolist()))) # read frequencies
  data.append(project)

print(data)

[['18000100', '180001-00', 0, [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], ['18111002', '181110-02', 52, [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], ['18111003', '181110-03', 0, [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], ['18111008', '181110-08', 0, [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], ['18111013', '181110-13', 0, [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], ['18111017', '181110-17', 0, [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], ['18111013', '181111-13', 0, [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], ['18112001', '181120-01', 865, [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], ['18112003', '181120-03', 5134, [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], ['18112018', '181120-18', 0, [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], ['18112021', '181120-21', 0, [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], ['18112022', '181120-22', 2736, [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], ['18112001', '181121-01', 339, [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], 

In [ ]:
write_unit_cost = [0.13, 0]
read_unit_cost = [0.013, 0]
resting_data_cost = [0.015, 0.192]


def calculate_cost(current_tier, action_tier, size, write_freq, read_freq):

  # calculate total storage cost
  storage_cost = (size*resting_data_cost[current_tier] / 1073.74) 

  # calculate total write cost
  cool_cost = 0
  if current_tier == 0:
    cool_cost = size*(0.01/1073.74)
  write_cost = write_freq * (write_unit_cost[current_tier] + cool_cost)

  # calculate total read cost
  cool_cost = 0
  if current_tier == 0: 
    cool_cost = size*(0.01/1073.74)
  read_cost = read_freq * (read_unit_cost[current_tier] + cool_cost)

  # calculate cost of possible tier change
  change_cost = 0
  if current_tier != action_tier: 
    cool_cost = 0
    if action_tier == 0:
        cool_cost = size*(0.01/1073.74)
    change_cost = read_unit_cost[current_tier] + write_unit_cost[action_tier] + cool_cost

  # calculate total cost and reward
  cost = storage_cost + write_cost + read_cost + change_cost

  return cost

In [ ]:
from stable_baselines3 import PPO
# Premium = 1
# Cool = 0
total_cost = 0
# load previously trained model
model = PPO.load("PPO_stable_baselines3_trained.zip")

# input state to predict action on
observation = np.array([1, 100, 0, 3])

for project in data:

  # input to cost method is [current_tier, size, write_freq, read_freq]
  current_tier = 1
  size = project[2]
  project_cost = 0

  # for loop to iterate through all days of week, calls method, returns cost for one day
  for i in range(7): 
    write_freq = project[3][i]
    read_freq = project[4][i]
    observation = np.array([current_tier, size, write_freq, read_freq])
    next_action = model.predict(observation, deterministic=True)[0] # or 0 or model prediction
    project_cost += calculate_cost(current_tier, next_action, size, write_freq, read_freq)
    current_tier = next_action
  total_cost += project_cost
  project.append(current_tier)

print(data)
print(total_cost)


[['18000100', '180001-00', 0, [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], 1, 0, 1, 0, 1], ['18111002', '181110-02', 52, [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], 0, 0, 1, 0, 0], ['18111003', '181110-03', 0, [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], 1, 0, 1, 0, 1], ['18111008', '181110-08', 0, [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], 1, 0, 1, 0, 1], ['18111013', '181110-13', 0, [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], 1, 0, 1, 0, 1], ['18111017', '181110-17', 0, [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], 1, 0, 1, 0, 1], ['18111013', '181111-13', 0, [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], 1, 0, 1, 0, 1], ['18112001', '181120-01', 865, [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], 0, 0, 1, 0, 0], ['18112003', '181120-03', 5134, [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], 0, 0, 1, 0, 0], ['18112018', '181120-18', 0, [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], 1, 0, 1, 0, 1], ['18112021', '181120-21', 0, [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], 1, 0, 1,

In [ ]:
import pandas as pd

projects = []
for project in data: 
  projects.append([project[0], project[1]])
print(projects)

df = pd.DataFrame(projects[:])
print(df)

df.to_csv('closed.csv')

[['18000100', '180001-00'], ['18111002', '181110-02'], ['18111003', '181110-03'], ['18111008', '181110-08'], ['18111013', '181110-13'], ['18111017', '181110-17'], ['18111013', '181111-13'], ['18112001', '181120-01'], ['18112003', '181120-03'], ['18112018', '181120-18'], ['18112021', '181120-21'], ['18112022', '181120-22'], ['18112001', '181121-01'], ['18112003', '181121-03'], ['18112107', '181121-07'], ['18112110', '181121-10'], ['18112114', '181121-14'], ['18112115', '181121-15'], ['18112018', '181121-18'], ['18112119', '181121-19'], ['18112021', '181121-21'], ['18112022', '181121-22'], ['18113003', '181130-03'], ['18113007', '181130-07'], ['18113008', '181130-08'], ['18113010', '181130-10'], ['18113011', '181130-11'], ['18113025', '181130-25'], ['18113029', '181130-29'], ['18113030', '181130-30'], ['18113007', '181131-07'], ['18113008', '181131-08'], ['18113121', '181131-21'], ['18113126', '181131-26'], ['18113029', '181131-29'], ['18113030', '181131-30'], ['18113024', '181132-24'], 